In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))])

In [3]:
training_data = torchvision.datasets.CIFAR10(root = './data',train=True,download=True,transform=transform)
test_data = torchvision.datasets.CIFAR10(root ='./data',train =True,download =True,transform=transform)


100%|██████████| 170498071/170498071 [00:02<00:00, 77138793.72it/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [4]:
batch_size =64
traindata_loader = torch.utils.data.DataLoader(training_data,batch_size=batch_size,shuffle=True)
testdata_loader = torch.utils.data.DataLoader(training_data,batch_size=batch_size,shuffle=True)

***Architecture***

In [5]:
class CNN(nn.Module):
    def __init__(self):
        
        super(CNN,self).__init__()
        self.conv1 = nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(6,16,5)
        self.fc1 = nn.Linear(16*10*10,120)
        self.fc2 = nn.Linear(120,80)
        self.fc3 = nn.Linear(80,10)
        
    def forward(self,x):
            #print(x.shape) 
            x = self.conv1(x)#input:3*32*32,#output:6*28*28
            print(x.shape)
            x = F.relu(x)
            x = self.pool(x)
            print(x.shape)
            x = self.conv2(x)#6*14*14
            print(x.shape) #16*10*10
            x = x.view(-1,16*10*10)
            x = self.fc1(x)
            x = F.relu(x)
            x = self.fc2(x)
            x = F.relu(x)
            x = self.fc3(x)
            return x
            
            

***Train the model***

In [6]:
model = CNN()

In [7]:
for (X,y) in traindata_loader:
    print(X.shape)
    print(X[0].shape)
    model(X[0])
    break

torch.Size([64, 3, 32, 32])
torch.Size([3, 32, 32])
torch.Size([6, 28, 28])
torch.Size([6, 14, 14])
torch.Size([16, 10, 10])


***Defining loss function***

In [8]:
cnn = CNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(),lr=0.001,momentum =0.9)

***Define Training data***

In [9]:
def train(traindata_loader,model,loss_fn,optimizer):
    model.train()
    size = len(traindata_loader.dataset)
    
    for(X,y) in traindata_loader:
        
        for epoch in range(epochs):
            pred = model(X)
            loss = loss_fn(pred,y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            print(f"{epochs+1}/{epochs},Train Loss: {loss.item():.3f}") 

***Define Test data***

In [10]:
def test(testdata_loader,model,loss_fn):
    model.eval()
    test_error,correct =0,0
    size = len(testdata_loader.dataset)
    
    with torch.no_grad():
        for(X,y) in testdata_loader:
            pred = model(X)
            loss =loss_fn(pred,y)
            test_loss = loss_fn(pred,y).item()
            test_error+=loss_fn(pred,y).item()
            correct += (pred.argmax(1)==y).type(torch.float).sum().item()
            print(f'test_correct {correct} out of {size} and  Accuracy is{correct/size*100}' )
                  

In [11]:
import time
start =time.time()

In [12]:
epochs =5
i=1
for t in range(epochs):
    print('-------epochs--------', i)
    train(traindata_loader,model,loss_fn,optimizer)
    test(testdata_loader,model,loss_fn)
    i+=1

-------epochs-------- 1
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.302
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.302
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.302
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.302
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.302
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.305
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.305
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.304
torch.Size([64, 6, 28, 28])
torch.Size([64, 6, 14, 14])
torch.Size([64, 16, 10, 10])
6/5,Train Loss: 2.304
torch.Size([6